## Applying Logistic Regression Model to 2017 Data ##

In [10]:
# Importing packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from scipy.stats import iqr
from sklearn import preprocessing
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [11]:
# Loading 2017 data
data = pd.read_csv("2017_Data.csv", header=0, index_col=0)
data.head()
data_concatinate = pd.read_csv("Concatinate_This.csv", header=0, index_col=0)

In [12]:
# Splitting data into target/data variables

data_year = data.Year
data_assigned = data.Assigned_ID
data_bank = data.Bank
data_mortdate = data.MortDate
data = data.drop(columns=['Year','Assigned_ID','Bank','MortDate'], axis=1)

In [13]:
# Checking data
data.head()

LTV_M  Purpose_M  Coop_M  Product_M  Occup_M  Back_M  BoCreditScor_M  \
0  2.348       1.61     1.0      0.879      1.0   1.000           0.743   
1  2.846       1.61     1.0      0.879      1.0   1.000           0.743   
2  2.846       1.00     1.0      0.879      1.0   1.431           0.743   
3  0.453       1.61     1.0      0.879      1.0   1.431           1.760   
4  1.526       1.61     1.0      0.879      1.0   2.020           2.850   

   PropType_M  Loan_Multiplier  
0         1.0         2.468890  
1         1.0         2.992530  
2         1.0         2.659820  
3         1.0         1.614601  
4         1.0        12.432702

#### Outliers ####

__No outliers for this data as everything is binned.__

#### Normalization ####

In [14]:
# Checking the mean of the data
data.mean()

LTV_M              1.187035
Purpose_M          1.250602
Coop_M             1.000000
Product_M          0.879000
Occup_M            1.006970
Back_M             1.479330
BoCreditScor_M     1.469742
PropType_M         1.011208
Loan_Multiplier    3.209812
dtype: float64

#### Skewness ####

In [15]:
# Checking skewness
data.skew()

LTV_M               0.815433
Purpose_M           0.362529
Coop_M              0.000000
Product_M           0.000000
Occup_M             6.551674
Back_M              0.601544
BoCreditScor_M      2.091029
PropType_M         10.975720
Loan_Multiplier     3.946952
dtype: float64

__There is no skewness, as the attributes that have high skew are categorical variables.__

In [16]:
# Concatinating the dataframe back together

data = pd.concat([data, data_year], axis=1, join='inner')
data = pd.concat([data, data_bank], axis=1, join='inner')
data = pd.concat([data, data_mortdate], axis=1, join='inner')
data = pd.concat([data, data_assigned], axis=1, join='inner')
data.head()

LTV_M  Purpose_M  Coop_M  Product_M  Occup_M  Back_M  BoCreditScor_M  \
0  2.348       1.61     1.0      0.879      1.0   1.000           0.743   
1  2.846       1.61     1.0      0.879      1.0   1.000           0.743   
2  2.846       1.00     1.0      0.879      1.0   1.431           0.743   
3  0.453       1.61     1.0      0.879      1.0   1.431           1.760   
4  1.526       1.61     1.0      0.879      1.0   2.020           2.850   

   PropType_M  Loan_Multiplier  Year     Bank  MortDate  Assigned_ID  
0         1.0         2.468890  2017  Atlanta      2016      2092102  
1         1.0         2.992530  2017  Atlanta      2016      2092103  
2         1.0         2.659820  2017  Atlanta      2016      2092104  
3         1.0         1.614601  2017  Atlanta      2016      2092105  
4         1.0        12.432702  2017  Atlanta      2016      2092106

### Loading Logistic Regression Model Already Built ###

In [17]:
# Loading logistic regression model
logreg = pickle.load(open('logreg_model.p','rb'))

In [18]:
# Attempt 1
pred_cols = list(data.iloc[:,[0,1,2,3,4,5,6,7]])

scaler = MinMaxScaler()
X_pred = scaler.fit(data[pred_cols]).transform(data[pred_cols])

predictions = pd.Series(logreg.predict(X_pred))
print(predictions.value_counts())

1    36128
0    19862
dtype: int64


In [19]:
data['Portfolio_Worthy'] = predictions
data.head()

LTV_M  Purpose_M  Coop_M  Product_M  Occup_M  Back_M  BoCreditScor_M  \
0  2.348       1.61     1.0      0.879      1.0   1.000           0.743   
1  2.846       1.61     1.0      0.879      1.0   1.000           0.743   
2  2.846       1.00     1.0      0.879      1.0   1.431           0.743   
3  0.453       1.61     1.0      0.879      1.0   1.431           1.760   
4  1.526       1.61     1.0      0.879      1.0   2.020           2.850   

   PropType_M  Loan_Multiplier  Year     Bank  MortDate  Assigned_ID  \
0         1.0         2.468890  2017  Atlanta      2016      2092102   
1         1.0         2.992530  2017  Atlanta      2016      2092103   
2         1.0         2.659820  2017  Atlanta      2016      2092104   
3         1.0         1.614601  2017  Atlanta      2016      2092105   
4         1.0        12.432702  2017  Atlanta      2016      2092106   

   Portfolio_Worthy  
0                 0  
1                 0  
2                 1  
3                 0  
4                 0

In [20]:
data_concatinate = data_concatinate.drop(columns=['LTV_M','Purpose_M','Coop_M','Product_M','Occup_M','Back','BoCreditScor_M','PropType_M','Year','Bank','MortDate','Assigned_ID','Back_M'])
data_concatinate.head()

IncRat     UPB   LTV  Purpose  Coop  Product  FedGuar  Occup   Front  \
0  3.2928   71967  0.89        2     2        1        2      1  0.1118   
1  4.0513  514837  0.94        2     2        1        2      1  0.1446   
2  2.2109  425563  1.04        1     2        1        2      1  0.2312   
3  2.3403  251800  0.75        2     2        1        2      1  0.1657   
4  1.0537  290613  0.85        2     2        1        2      1  0.4062   

   BoCreditScor PropType  Loan_Multiplier  
0             5     PT01         2.468890  
1             5     PT01         2.992530  
2             5     PT01         2.659820  
3             4     PT07         1.614601  
4             3     PT01        12.432702

In [21]:
# Concatinating Everything together
data = pd.concat([data,data_concatinate], axis=1, join='inner')
data.head()

LTV_M  Purpose_M  Coop_M  Product_M  Occup_M  Back_M  BoCreditScor_M  \
0  2.348       1.61     1.0      0.879      1.0   1.000           0.743   
1  2.846       1.61     1.0      0.879      1.0   1.000           0.743   
2  2.846       1.00     1.0      0.879      1.0   1.431           0.743   
3  0.453       1.61     1.0      0.879      1.0   1.431           1.760   
4  1.526       1.61     1.0      0.879      1.0   2.020           2.850   

   PropType_M  Loan_Multiplier  Year       ...          LTV  Purpose  Coop  \
0         1.0         2.468890  2017       ...         0.89        2     2   
1         1.0         2.992530  2017       ...         0.94        2     2   
2         1.0         2.659820  2017       ...         1.04        1     2   
3         1.0         1.614601  2017       ...         0.75        2     2   
4         1.0        12.432702  2017       ...         0.85        2     2   

   Product  FedGuar  Occup   Front  BoCreditScor  PropType  Loan_Multiplier  
0        1        2      1  0.1118             5      PT01         2.468890  
1        1        2      1  0.1446             5      PT01         2.992530  
2        1        2      1  0.2312             5      PT01         2.659820  
3        1        2      1  0.1657             4      PT07         1.614601  
4        1        2      1  0.4062             3      PT01        12.432702  

[5 rows x 26 columns]

In [22]:
data.to_excel("2017_Good_Data.xlsx")